In [2]:
class A_Polynomial:
    ''' Abstract Polynomial Base Class'''
    def get_degree(self):
        ''' returns leading coefficient '''
        val_str = bin(self.val)
        # remove 0b prefix from string
        val_str = val_str[2:]
        for i in range(len(val_str)):
            if(val_str[i] == '1'):
                return len(val_str)-1 - i
        return 0
        
    def __str__(self):
        result = ''
        # trim leading 0b
        str_val = bin(self.val)[2:]
        for i, val in enumerate(str_val):
            cur_degree = self.get_degree()-i
            if val == '1':
                result += f'x^{cur_degree} +' 
        # trim leading plus
        result = result[0:-1]
        return result + f', bin: {bin(self.val)}, dec: {self.val}'
    


In [3]:
class IrreducablePolynomial(A_Polynomial):
    irreducable_polys = {
        4: 0b111,
        8: 0b1101,
        16: 0b11001
    }
    def __init__(self, field_num: int):
        ''' field_num - size of the field we want to use'''
        self.val = self.irreducable_polys.get(field_num, -1)
        if self.val == -1:
            raise Exception('No Binary Field of That Size Exists.')
        self.degree = None

In [54]:
from copy import deepcopy
from math import log

class Polynomial(A_Polynomial):
    def __init__(self, val: int, poly: IrreducablePolynomial, log=False):
        '''
            polynomial mod 2
            val - value of this polynomial as bit string
            poly - irreducible polynomial to mod by
        '''
        self.val = val
        self.poly = poly
        self.log = log
        self.degree = None
    
    def __add__(self, other):
        ''' xor this val with others val then mod by poly'''
        result = self.val ^ other.val
        return Polynomial(result, self.poly) % self.poly
    
    def __mul__(self, other):
        product = 0
        str_val = bin(other.val)[2:]
        for i, val in enumerate(str_val):
            if val == "1":
                shift = len(str_val)-1-i
                val_shifted = self.val << shift
                product ^= val_shifted
        #print(f'product: {bin(product)}')
        return Polynomial(product, poly=self.poly) % self.poly
    
    def pow_mod(self, base, exp, p, b):
        '''computes base**exp % p'''
        if exp==1:
            ans = base*b % p
            print(type(ans))
            return ans
        elif exp%2==0:
            return self.pow_mod(base*base % p, exp//2, p, b)
        return self.pow_mod(base*base % p, (exp-1)//2, p, base*b % p)
    
    def __pow__(self, power):
#         product = self
#         for i in range(1,power):
#             product *= self
#         return product
        return self.pow_mod(base=self, exp=power, p=self.poly, b=Polynomial(0b1, self.poly))

    def __mod__(self, other: IrreducablePolynomial):
        ''' other is an irreducable polynomial'''
        this = deepcopy(self)
        other = deepcopy(other)
        #print(f'this: {this}, irreducable: {other}')
        while(this.get_degree() >= other.get_degree()):
            #print(f'this_degree: {this.get_degree()}, irreducabledeg: {other.get_degree()}')
            diff_degree = this.get_degree() - other.get_degree()
            #print(f'this: {this}, irreducable: {other}')
            #print(f'diff_degree: {diff_degree}')
            #print(f'old_val: {this.val}')
            #print(f'{bin(this.val)} ^ {bin(other.val<<diff_degree)}')
            this.val = this.val ^ (other.val<<diff_degree)
            #print(f'new_val: {this.val}\n')
        return Polynomial(this.val, self.poly)
    def frobenius(self):
        return self**2
#     def trace(self):
#         tr(8) in F16 = 8 + 8^2 + 8^4 + 8^8
#         times = log(self.poly.val, 2)
#         times = int(times)-1
#         print(times)
#         res_sum = self
#         result = self
#         for i in range(times):
#             res_sum += result
#             result = result.frobenius()
#         return res_sum

In [55]:
# test get_leading and add
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b100101,poly=irred)
p2 = Polynomial(val=0b100,poly=irred)
print(f'p1-> degree of {p1}')
print(f'p2-> degree of {p2}')
print(f'p1 + p2 % F16= {p1 + p2}')

p1-> degree of x^5 +x^2 +x^0 , bin: 0b100101, dec: 37
p2-> degree of x^2 , bin: 0b100, dec: 4
p1 + p2 % F16= x^3 +x^1 , bin: 0b1010, dec: 10


In [56]:
# mod test
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b1111000,poly=irred)
#print(f'p1-> degree of {p1}: {p1.get_degree()}')
#print(f'p2-> degree of {p2}: {p2.get_degree()}')
print(f'p1 % irred = {p1 % p1.poly}')

p1 % irred = x^2 +x^0 , bin: 0b101, dec: 5


In [57]:
# test multiply
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b1000,poly=irred)
p2 = Polynomial(val=0b1111,poly=irred)
print(f'p1 * p2 = {p1 * p2}')

p1 * p2 = x^2 +x^0 , bin: 0b101, dec: 5


In [58]:
# test power
p1 = Polynomial(val=0b1000,poly=irred)
print(p1**2)

type of poly: <class '__main__.IrreducablePolynomial'>, type of exp: <class 'int'>, type of p:<class '__main__.IrreducablePolynomial'>, type of b: <class '__main__.Polynomial'>
type of poly: <class '__main__.IrreducablePolynomial'>, type of exp: <class 'int'>, type of p:<class '__main__.IrreducablePolynomial'>, type of b: <class '__main__.Polynomial'>
<class '__main__.Polynomial'>
x^3 +x^2 +x^1 +x^0 , bin: 0b1111, dec: 15


In [8]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b1000, poly=irred)
print(p1.frobenius())

x^3 +x^2 +x^1 +x^0 , bin: 0b1111, dec: 15


In [9]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b1011,poly=irred)
p2 = Polynomial(val=0b1111,poly=irred)
print(f'p1 * p2 = {p1 * p2}')

p1 * p2 = x^3 +x^2 +x^0 , bin: 0b1101, dec: 13


## Squares

In [33]:
def print_squares(p):
    l = []
    for i in range(0, (p-1)//2 + 1):
        square = pow(i, 2, p)
        pos_item = (i, square)
        l.append(pos_item)
        if i != 0:
            neg_item = (p-i, square)
            l.append(neg_item)
    squares = set([item[1] for item in l])
    print(f'squares mod {p}: {squares}\n')
    
    square_roots = dict(zip(squares, []*len(squares)))
    for num, square in l:
        if not square_roots.get(square):
            square_roots[square] = []
        square_roots[square].append(num)

    print('dict mapping squares --> list of square roots')
    for square, roots in square_roots.items():
        print(f'{square:2}: {roots}')
    return square_roots

In [34]:
print_squares(13);

squares mod 13: {0, 1, 3, 4, 9, 10, 12}

dict mapping squares --> list of square roots
 0: [0]
 1: [1, 12]
 4: [2, 11]
 9: [3, 10]
 3: [4, 9]
12: [5, 8]
10: [6, 7]


## Fermat Factorization

In [12]:
from math import sqrt
def get_factor(n):
    #(y, sqrt(n+y**2)) where second expression is an integer
    pair = (0,0)
    for y in range(1,21):
        item = (y, sqrt(n+y**2))
        val = item[1]
        if(int(val) == val):
            pair = item
            break
    if pair == (0,0):
        raise Exception('no factor found')
    y, x = pair
    factors = (int(x+y), int(x-y))
    return factors

In [13]:
n = 62_251_979
get_factor(n)

(7901, 7879)

In [14]:
# Quadratic - two solutions if trace(ac/b^2) = 0, b != 0
# Cyclic - 2**(k) for k 0,...,p-1 in F_p

## Midterm

### Problem 1

#### a.)

In [15]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=9, poly=irred)
p2 = Polynomial(val=13, poly=irred)
print(p1+p2)

x^2 , bin: 0b100, dec: 4


#### b.)

In [16]:
print(p1*p2)

x^0 , bin: 0b1, dec: 1


#### c.)

In [17]:
p3 = Polynomial(val=7, poly=irred)
print(p3**5)

x^3 +x^1 +x^0 , bin: 0b1011, dec: 11


In [18]:
print(p3)
print(p3**2)
print(p3**4)
print(p3**5)

x^2 +x^1 +x^0 , bin: 0b111, dec: 7
x^3 +x^2 , bin: 0b1100, dec: 12
x^2 +x^1 , bin: 0b110, dec: 6
x^3 +x^1 +x^0 , bin: 0b1011, dec: 11


### Problem 2

#### a.)

In [20]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=6, poly=irred)
p2 = Polynomial(val=5, poly=irred)
print((p1+p2)**2)

x^2 +x^0 , bin: 0b101, dec: 5


#### b.)

In [21]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=4, poly=irred)
p2 = Polynomial(val=5, poly=irred)
print(p1**2)
print(p2**2)
print((p1+p2)**2)

x^3 +x^0 , bin: 0b1001, dec: 9
x^3 , bin: 0b1000, dec: 8
x^0 , bin: 0b1, dec: 1


#### c.)

In [22]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=15, poly=irred)
print(p1**8)

x^3 , bin: 0b1000, dec: 8


In [23]:
print(p1**2)
print(p1**4)
print(p1**8)

x^1 +x^0 , bin: 0b11, dec: 3
x^2 +x^0 , bin: 0b101, dec: 5
x^3 , bin: 0b1000, dec: 8


In [24]:
p1 = Polynomial(val=3, poly=irred)
print(p1**2)

x^2 +x^0 , bin: 0b101, dec: 5


### Problem 3

#### a.)

In [55]:
square_dict = print_squares(11)
square_dict

squares mod 11: {0, 1, 3, 4, 5, 9}

dict mapping squares --> list of square roots
 0: [0]
 1: [1, 10]
 4: [2, 9]
 9: [3, 8]
 5: [4, 7]
 3: [5, 6]


{0: [0], 1: [1, 10], 4: [2, 9], 9: [3, 8], 5: [4, 7], 3: [5, 6]}

#### b.)

In [56]:
l = []
for x in range(11):
    cur_val = (x**3 + 5*x + 2) % 11
    if cur_val not in square_dict.keys():
        continue
    y = square_dict[cur_val]
    for root in y:
        l.append((x, root))

In [57]:
[print(item) for item in l];

(2, 3)
(2, 8)
(3, 0)
(4, 3)
(4, 8)
(5, 3)
(5, 8)
(8, 2)
(8, 9)


In [58]:
len(l)

9

### Problem 4

In [59]:
get_factor(1476221)

(1217, 1213)

In [60]:
1217*1213

1476221